# Semantic Kernel

The Semantic Kernel, initially created by Microsoft as an open-source SDK, serves as the foundational technology for Microsoft 365 Copilot and Bing, with the primary goal of simplifying the integration of Large Language Models (LLMs) into various applications. This SDK empowers users to harness the capabilities of LLMs for orchestrating workflows through natural language queries and commands. This is achieved by facilitating seamless connections between these language models and external services, which offer supplementary functionalities essential for task completion.

https://github.com/microsoft/semantic-kernel

> Reference: https://towardsdatascience.com/a-pythonistas-intro-to-semantic-kernel-af5a1a39564d

In [1]:
# %pip install semantic-kernel

In [2]:
import datetime
import openai
import os
import semantic_kernel as sk
import sys

from dotenv import load_dotenv
from semantic_kernel.connectors.ai.open_ai import (
    AzureChatCompletion,
    AzureTextCompletion,
)
from semantic_kernel.connectors.ai.hugging_face import HuggingFaceTextCompletion
from semantic_kernel import PromptTemplateConfig
from semantic_kernel import PromptTemplate
from semantic_kernel import SemanticFunctionConfig
from semantic_kernel.connectors.ai.open_ai import AzureTextEmbedding
from semantic_kernel.text import text_chunker as tc

In [3]:
sys.version

'3.10.10 (main, Mar 21 2023, 18:45:11) [GCC 11.2.0]'

In [4]:
print(f"Today is {datetime.datetime.today().strftime('%d-%b-%Y %H:%M:%S')}")

Today is 23-Oct-2023 09:23:53


In [5]:
load_dotenv("azure.env")

# Azure Open AI
openai.api_type: str = "azure"
openai.api_key = os.getenv("OPENAI_API_KEY")
openai.api_base = os.getenv("OPENAI_API_BASE")
openai.api_version = os.getenv("OPENAI_API_VERSION")

print("Open AI version:", openai.__version__)

Open AI version: 0.28.1


In [6]:
OPENAI_DEPLOYMENT_NAME = "text-davinci-003"

In [7]:
def print_ai_services(kernel):
    print(f"Text completion services: {kernel.all_text_completion_services()}")
    print(f"Chat completion services: {kernel.all_chat_services()}")
    print(
        f"Text embedding generation services: {kernel.all_text_embedding_generation_services()}"
    )

## Semantic kernel

In [8]:
kernel = sk.Kernel()

In [9]:
kernel.add_text_completion_service(
    service_id="azure_gpt35_text_completion",
    service=AzureTextCompletion(
        OPENAI_DEPLOYMENT_NAME, openai.api_base, openai.api_key
    ),
)

gpt35_chat_service = AzureChatCompletion(
    deployment_name=OPENAI_DEPLOYMENT_NAME,
    endpoint=openai.api_base,
    api_key=openai.api_key,
)

kernel.add_chat_service("azure_gpt35_chat_completion", gpt35_chat_service)

In [10]:
print_ai_services(kernel)

Text completion services: ['azure_gpt35_text_completion', 'azure_gpt35_chat_completion']
Chat completion services: ['azure_gpt35_chat_completion']
Text embedding generation services: []


In [11]:
prompt = """
{{$input}} is the capital city of
"""

generate_capital_city_text = kernel.create_semantic_function(
    prompt, max_tokens=100, temperature=0, top_p=0
)

In [12]:
response = generate_capital_city_text("Paris")
response

SKContext(memory=<semantic_kernel.memory.null_memory.NullMemory object at 0x7f80a0483e20>, variables=ContextVariables(variables={'input': 'France.'}), skill_collection=ReadOnlySkillCollection(data={'_global_functions_': {'f_8ae11441_9dea_4dee_ab9e_40772af3267b': <semantic_kernel.orchestration.sk_function.SKFunction object at 0x7f80a00269b0>}}))

In [13]:
response = await generate_capital_city_text.invoke_async("Paris")

In [14]:
response.result

'France.'

In [15]:
print(response)

France.


## Using GPT2 from HuggingFace

In [16]:
hf_model = HuggingFaceTextCompletion("gpt2", task="text-generation")
kernel.add_text_completion_service("hf_gpt2_text_completion", hf_model)

Xformers is not installed correctly. If you want to use memorry_efficient_attention to accelerate training use the following command to install Xformers
pip install xformers.


In [17]:
hf_config_dict = {
    "schema": 1,
    # The type of prompt
    "type": "completion",
    # A description of what the semantic function does
    "description": "Provides information about a capital city, which is given as an input, using the GPT2 model",
    # Specifies which model service(s) to use
    "default_services": ["hf_gpt2_text_completion"],
    # The parameters that will be passed to the connector and model service
    "completion": {
        "temperature": 0.7,
        "top_p": 1,
        "max_tokens": 1000,
        "number_of_responses": 1,
    },
    # Defines the variables that are used inside of the prompt
    "input": {
        "parameters": [
            {
                "name": "input",
                "description": "The name of the capital city",
                "defaultValue": "London",
            }
        ]
    },
}

In [18]:
hf_config_dict

{'schema': 1,
 'type': 'completion',
 'description': 'Provides information about a capital city, which is given as an input, using the GPT2 model',
 'default_services': ['hf_gpt2_text_completion'],
 'completion': {'temperature': 0.7,
  'top_p': 1,
  'max_tokens': 1000,
  'number_of_responses': 1},
 'input': {'parameters': [{'name': 'input',
    'description': 'The name of the capital city',
    'defaultValue': 'London'}]}}

In [19]:
prompt_template_config = PromptTemplateConfig.from_dict(hf_config_dict)

In [20]:
prompt_template = sk.PromptTemplate(
    template="{{$input}} is the capital city of",
    prompt_config=prompt_template_config,
    template_engine=kernel.prompt_template_engine,
)

In [21]:
function_config = SemanticFunctionConfig(prompt_template_config, prompt_template)

In [22]:
def create_semantic_function_config(prompt_template, prompt_config_dict, kernel):
    prompt_template_config = PromptTemplateConfig.from_dict(prompt_config_dict)
    prompt_template = sk.PromptTemplate(
        template=prompt_template,
        prompt_config=prompt_template_config,
        template_engine=kernel.prompt_template_engine,
    )
    return SemanticFunctionConfig(prompt_template_config, prompt_template)

In [23]:
gpt2_complete = kernel.register_semantic_function(
    skill_name="GPT2Complete",
    function_name="gpt2_complete",
    function_config=create_semantic_function_config(
        "{{$input}} is a city of", hf_config_dict, kernel
    ),
)

In [24]:
response = gpt2_complete("Roma")

In [25]:
print(response)

Roma is a city of 600 square miles in the heart of northern California. The city is the largest in the United States with 1.2 million residents.

The city has been on the list of the top 10 worst cities in the United States for several years now, but the 2014 survey of 1,000 residents found the city ranked third worst.

The United States ranked fourth in the world for the worst rates of poverty among the top 10 most deprived cities in the world.

The median household income in San Francisco is just $48,000, but the median annual incomes in the city are $50,000.

In its 2013 report, the San Francisco Department of Finance said that it had received more than 100 requests for information about the city's most deprived areas, such as the homeless, for the 2013 Census.

An independent study conducted by the San Francisco Public Library found that the median home price in San Francisco was $106,000 in 2013, compared to $81,000 in 2012.

The average price of a house in San Francisco was $3,00

In [26]:
response = gpt2_complete("Paris?")
print(response)

Paris? is a city of more than 200 million people. And so, there's a lot to do and find.

In the past year or so, I've been lucky enough to spend time with a lot of people from all around the world. On the road, I've been lucky enough to meet some of the best people in the world.

But the last few years have been really tough.

That's why it's so important to look at the past, not look back.

For me, the last few years have been really tough.

But I know what it's like to live with a lot of loss and to lose everything.

There's really no other way.

I feel so lucky to have people like you who have come through the difficult times.

I also feel so thankful to have such a strong team with so many people.

And I feel so grateful to have so many people who are here and doing this work every day, and to be able to give back.

The more I look back at the past, the more I feel like I'm part of something bigger than what I did.

I think about all the people who have been here and who have been 

## Chatbot

In [27]:
chat_config_dict = {
    "schema": 1,
    # The type of prompt
    "type": "completion",
    # A description of what the semantic function does
    "description": "Provides information about a capital city, which is given as an input, using the GPT3.5 model",
    # Specifies which model service(s) to use
    "default_services": ["azure_gpt35_chat_completion"],
    # The parameters that will be passed to the connector and model service
    "completion": {
        "temperature": 0.0,
        "top_p": 1,
        "max_tokens": 256,
        "number_of_responses": 1,
        "presence_penalty": 0,
        "frequency_penalty": 0,
    },
    # Defines the variables that are used inside of the prompt
    "input": {
        "parameters": [
            {
                "name": "input",
                "description": "The name of the capital city",
                "defaultValue": "London",
            }
        ]
    },
}

capital_city_chat = kernel.register_semantic_function(
    skill_name="CapitalCityChat",
    function_name="capital_city_chat",
    function_config=create_semantic_function_config(
        "{{$input}} is the capital city of", chat_config_dict, kernel
    ),
)

In [28]:
response = capital_city_chat("Paris")
response.result

'Paris'

In [29]:
kernel.add_text_completion_service(
    service_id="azure_gpt35_text_completion",
    service=AzureTextCompletion(
        OPENAI_DEPLOYMENT_NAME, openai.api_base, openai.api_key
    ),
)

gpt35_chat_service = AzureChatCompletion(
    deployment_name="gpt-35-turbo-16k",
    endpoint=openai.api_base,
    api_key=openai.api_key,
)

kernel.add_chat_service("azure_gpt35_chat_completion", gpt35_chat_service)

In [30]:
chatbot = kernel.register_semantic_function(
    skill_name="Chatbot",
    function_name="chatbot",
    function_config=create_semantic_function_config(
        "{{$input}}", chat_config_dict, kernel
    ),
)

In [31]:
print(chatbot("Paris is the capital of which country?"))

Paris is the capital of France.


In [32]:
print(chatbot("What to visit in Paris?"))

There are numerous attractions to visit in Paris. Some popular ones include:

1. Eiffel Tower: The iconic symbol of Paris, offering stunning views of the city from its observation decks.

2. Louvre Museum: One of the world's largest and most visited museums, housing famous artworks like the Mona Lisa and Venus de Milo.

3. Notre-Dame Cathedral: A masterpiece of Gothic architecture, known for its stunning stained glass windows and gargoyles.

4. Montmartre: A historic neighborhood famous for its bohemian atmosphere, art studios, and the beautiful Sacré-Cœur Basilica.

5. Champs-Élysées: A famous avenue lined with shops, cafes, and theaters, leading to the Arc de Triomphe.

6. Palace of Versailles: Located just outside Paris, this opulent palace was the residence of French kings and queens, known for its stunning gardens.

7. Sainte-Chapelle: A breathtaking medieval chapel renowned for its stunning stained glass windows depicting biblical scenes.

8. Musée d'Orsay: Housed in a former rai

In [33]:
chatbot_prompt = """
"You are a chatbot to provide information about different cities and countries. 
 For other questions not related to places, you should politely decline to answer the question, stating your purpose"
 +++++

{{$history}}
User: {{$input}}
ChatBot: """

In [34]:
chat_config_dict = {
    "schema": 1,
    # The type of prompt
    "type": "completion",
    # A description of what the semantic function does
    "description": "A chatbot to provide information about cities and countries",
    # Specifies which model service(s) to use
    "default_services": ["azure_gpt35_chat_completion"],
    # The parameters that will be passed to the connector and model service
    "completion": {
        "temperature": 0.7,
        "top_p": 1,
        "max_tokens": 1000,
        "number_of_responses": 1,
        "presence_penalty": 0,
        "frequency_penalty": 0,
    },
    # Defines the variables that are used inside of the prompt
    "input": {
        "parameters": [
            {
                "name": "input",
                "description": "The input given by the user",
                "defaultValue": "",
            },
            {
                "name": "history",
                "description": "Previous interactions between the user and chatbot",
                "defaultValue": "",
            },
        ]
    },
}

In [35]:
function_config = create_semantic_function_config(
    chatbot_prompt, chat_config_dict, kernel
)
chatbot = kernel.register_semantic_function(
    skill_name="SimpleChatbot",
    function_name="simple_chatbot",
    function_config=function_config,
)

In [36]:
context = kernel.create_new_context()
context["history"] = ""

In [37]:
async def chat(input_text, context, verbose=True):
    # Save new message in the context variables
    context["input"] = input_text

    if verbose:
        # print the full prompt before each interaction
        print("Prompt:")
        print("-----")
        # inject the variables into our prompt
        print(await function_config.prompt_template.render_async(context))
        print("-----")

    # Process the user message and get an answer
    answer = await chatbot.invoke_async(context=context)

    # Show the response
    print(f"ChatBot: {answer}")

    # Append the new interaction to the chat history
    context["history"] += f"\nUser: {input_text}\nChatBot: {answer}\n"

In [38]:
await chat("Paris is the capital of which country?", context)

Prompt:
-----

"You are a chatbot to provide information about different cities and countries. 
 For other questions not related to places, you should politely decline to answer the question, stating your purpose"
 +++++


User: Paris is the capital of which country?
ChatBot: 
-----
ChatBot: Paris is the capital of France.


In [39]:
await chat("What to visit there?", context)

Prompt:
-----

"You are a chatbot to provide information about different cities and countries. 
 For other questions not related to places, you should politely decline to answer the question, stating your purpose"
 +++++


User: Paris is the capital of which country?
ChatBot: Paris is the capital of France.

User: What to visit there?
ChatBot: 
-----
ChatBot: There are many famous attractions in Paris, including the Eiffel Tower, Louvre Museum, Notre-Dame Cathedral, and the Champs-Élysées. These are just a few examples of the numerous sites worth visiting in the city.


In [40]:
await chat("What are the main train stations?", context)

Prompt:
-----

"You are a chatbot to provide information about different cities and countries. 
 For other questions not related to places, you should politely decline to answer the question, stating your purpose"
 +++++


User: Paris is the capital of which country?
ChatBot: Paris is the capital of France.

User: What to visit there?
ChatBot: There are many famous attractions in Paris, including the Eiffel Tower, Louvre Museum, Notre-Dame Cathedral, and the Champs-Élysées. These are just a few examples of the numerous sites worth visiting in the city.

User: What are the main train stations?
ChatBot: 
-----
ChatBot: Paris has several main train stations, including Gare du Nord, Gare de Lyon, Gare de l'Est, Gare Montparnasse, and Gare Saint-Lazare. These stations connect Paris to various domestic and international destinations.


In [41]:
kernel.add_text_embedding_generation_service(
    "azure_openai_embedding",
    AzureTextEmbedding(
        deployment_name="text-embedding-ada-002",
        endpoint=openai.api_base,
        api_key=openai.api_key,
    ),
)

In [42]:
response = chatbot(
    """Please provide a comprehensive overview of things to do in London. Structure your answer in 5 paragraphs, based on:
- overview
- landmarks
- history
- culture
- food

Each paragraph should be 100 tokens, do not add titles such as 'Overview:'' or 'Food:'' to the paragraphs in your response.

Do not acknowledge the question, with a statement like "Certainly, here's a comprehensive overview of things to do in London". 
Do not provide a closing comment.
"""
)

In [43]:
london_info = response.result
print(london_info)

Certainly, here's a comprehensive overview of things to do in London. 

London, the capital city of England, is a vibrant and diverse metropolis with a rich history and a plethora of attractions. With its iconic landmarks, historical significance, diverse culture, and delectable food scene, there is something for everyone to enjoy in this bustling city.

One of the must-see landmarks in London is the Tower of London, a historic castle that has served as a royal palace, prison, and treasury. Visitors can explore the medieval architecture, view the Crown Jewels, and even witness the famous Beefeaters during the guided tours. Another iconic landmark is the Buckingham Palace, the official residence of the Queen. Witnessing the Changing of the Guard ceremony is a popular attraction for tourists.

London's history can be traced back to Roman times, and its rich past is evident throughout the city. The British Museum houses a vast collection of artifacts from various civilizations, including 

In [44]:
memory_store = sk.memory.VolatileMemoryStore()
kernel.register_memory_store(memory_store=memory_store)

In [45]:
chunks = tc.split_plaintext_paragraph([london_info], max_tokens=100)

In [46]:
for i, chunk in enumerate(chunks):
    await kernel.memory.save_information_async(
        collection="London", id="chunk" + str(i), text=chunk
    )

In [47]:
results = await kernel.memory.search_async(
    "London", "what food should I eat in London?", limit=2
)

for result in results:
    print(f"Text: {result.text} \nRelevance:{result. relevance}\n")

Text: When it comes to food, London is a paradise for food enthusiasts. From traditional English pubs serving hearty fare like fish and chips to Michelin-starred restaurants offering innovative cuisines, the city has it all. 
Relevance:0.8812976659374058

Text: Borough Market is a food lover's haven, with its wide range of fresh produce, artisanal goods, and international delicacies. Don't forget to try the quintessential British afternoon tea experience, complete with scones, clotted cream, and a selection of teas. 
Relevance:0.8383076237176478



In [48]:
results = await kernel.memory.search_async(
    "London", "Where can I eat non-british food in London?", limit=2
)
for result in results:
    print(f"Text: {result.text} \n\nRelevance = {result.relevance}\n")

Text: When it comes to food, London is a paradise for food enthusiasts. From traditional English pubs serving hearty fare like fish and chips to Michelin-starred restaurants offering innovative cuisines, the city has it all. 

Relevance = 0.8615324725854946

Text: Borough Market is a food lover's haven, with its wide range of fresh produce, artisanal goods, and international delicacies. Don't forget to try the quintessential British afternoon tea experience, complete with scones, clotted cream, and a selection of teas. 

Relevance = 0.8294832022254006

